<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    <h3>Release Planning Sample Workflow</h3>
    The following is an overview of the workflow:
    <ul>
    <li><b>Release Planning</b>: Prepares backlog of issues.</li>
    <li><b>Sprint Planning Cycle</b>: Scopes out sprint (includes Go/No-Go Decision for readiness. If readiness passes, signals kickoff of      next backlog issue, or ENDS if there are no more issues.) Also finetunes sprint tasks by preparing technical documents and plans which will be used as "goldens" for the sprint.</li>
    <li><b>Sprint Implementation Cycle</b>: Iterative implementation of the code,
    documents and/or tests that were indicated/"specified" by the specs in
    the planning phase.</li>
    <li><b>Sprint Evaluation Cycle</b>: Evaluates the code, documents, tests etc.
    created during the implementation cycle.
    </li>
    <li><b>Release Engineering</b>: Prepares the artifacts for readiness review.</li>
    <li><b>Release Delivery</b>: Delivers release.
    </li>
    </ul>
</div>

## Release Initiation / Kickoff

In [1]:
##############################################################################
# Imports
##############################################################################
import importlib
from crewai import Agent, Task, Crew, Process, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List, Optional
from pydantic import Field, BaseModel
from crewai_tools import SerperDevTool
from crewai.tools import tool
from crewai.flow.flow import Flow, listen, start, router
import os
from urllib.parse import urlparse
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), "..", ".."))
from tools import github_util
import logging
import traceback
import shutil
import time
from templates.crew import ReleaseCycle #, SprintCycle
logging.basicConfig(level=logging.INFO)
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
##########################################################
# Instance Variables
##########################################################
_GITHUB_PROJECT = os.getenv("GITHUB_PROJECT")

_ORIGINAL_SPEC = os.getenv("ORIGINAL_SPEC")

_FEATURE_BRANCH = "template"

_BASE_BRANCH = "refactored"

_LOCAL_PATH = "tmp"


In [3]:
class ReleaseInitiationFlow(Flow):
    """Flow for ReleaseInitiation."""

    def __init__(self):
        super().__init__()
        
        self.state['retries'] = 0
        
        self.max_depth = 10
    
    @start()
    def setup_environment(self):

        logging.info(f"Starting flow {self.state['id']}...cloning codebase and creating new feature branch...")
        
        github_util.clone_repo(branch=_BASE_BRANCH, local_path=_LOCAL_PATH)
        
        # github_util.create_feature_branch(_FEATURE_BRANCH, local_path=_LOCAL_PATH)
        
    
    @listen(setup_environment)
    def start_release_planning(self):

        logging.info(f"Starting flow {self.state['id']}...checking if release has any issues...")

        has_no_issues = github_util.is_project_empty(_GITHUB_PROJECT)

        if has_no_issues:

            logging.info(f"Board has no issues. Creating user stories...")

            additional_context = self.state['additional_context']

            if additional_context:

                additional_context = f"For additional context, read from this single file:\n{additional_context}"
                

            output = ReleaseCycle(_FEATURE_BRANCH, "REFERENCE").crew().kickoff(inputs={"input": self.state['input'],

                                                            "concatenated_output_file": self.state['concatenated_output_file'],

                                                            "additional_context": additional_context,

                                                            "output_base_path": f"{_LOCAL_PATH}/spec"})

            return output


    @listen(start_release_planning)
    def get_sprint_blocked_status(self, spec):

        logging.info(f"Starting flow {self.state['id']}...checking if sprint is blocked...")

        self.state['is_blocked'] = github_util.is_sprint_blocked(_GITHUB_PROJECT)
        
        self.state['is_in_progress'] = github_util.is_sprint_in_progress(_GITHUB_PROJECT)
        

    @listen(get_sprint_blocked_status)
    def get_sprint_next_step(self, result):

        if self.state['is_blocked']:

            return "end"

        if self.state['is_in_progress']:

            self.state['retries'] += 1

            if self.state['retries'] == self.max_depth:

                logging.info("Retries exhausted; exiting.")

                return "end"

            else:

                retries_left = self.max_depth - self.state['retries']

                logging.info(f"Existing issue is being processed. Checking progress again in 60 seconds - {retries_left} tries left...")

                time.sleep(60)

                return self.get_sprint_blocked_status()
                             
        else:

            logging.info("No existing issue is being processed. Processing the next issue in the backlog...")

            return "continue"

                
    @listen("continue")
    def process_next_story(self):

        logging.info("In continue.")

                
    @listen("end")
    def end(self):

        logging.info("Ending workflow.")

### Execute Code Translation Flow
Execute the flow!

In [4]:
##############################################################################
# Execute the Flow
##############################################################################

flow = ReleaseInitiationFlow()

flow.plot("ReleaseInitiationFlowPlot")

result = flow.kickoff(inputs={"input": f"{_LOCAL_PATH}/docs/document_list.txt",
                              "concatenated_output_file": f"{_LOCAL_PATH}/spec/concatenated.txt",
                              "additional_context": f"{_LOCAL_PATH}/README.md"})

logging.info("Release Initialization flow complete.")

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: ReleaseInitiationFlow                                                                                    │
│  ID: ac413622-8e6b-4966-94b5-23e7f6f7b47c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: ac413622-8e6b-4966-94b5-23e7f6f7b47c

INFO:crewai.flow.flow:Flow started with ID: ac413622-8e6b-4966-94b5-23e7f6f7b47c


/opt/miniconda3/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

INFO:root:Starting flow ac413622-8e6b-4966-94b5-23e7f6f7b47c...cloning codebase and creating new feature branch...
INFO:root:Cloning repo to tmp...
INFO:root:Repo tmp already exists. Removing...
INFO:root:Repo name: agapebondservant/industrial-data-project


...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5da30c41-ac9e-4ae4-b618-3964dffb900f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: <|start|>assistant<|channel|>analysis to=Read Files By Pattern                                   │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read Files By Pattern                 │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|channel|>analysis to=Read File By Name                     │
│  code<|message|><|call|>assistant<|start|>assistant<|c

/opt/miniconda3/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

INFO:root:Starting flow ac413622-8e6b-4966-94b5-23e7f6f7b47c...checking if sprint is blocked...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:No issues with status 'Ready' or 'Backlog' in project 'Release 1'.
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Project 'Release 1' has 0 active issue(s).


╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: ReleaseInitiationFlow                                                                                    │
│  ID: ac413622-8e6b-4966-94b5-23e7f6f7b47c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO:root:Release Initialization flow complete.


## Planning Cycle

## Implementation Cycle

## Evaluation Cycle

## Readiness Review Cycle

## Sprint Completion Cycle